In [22]:
import pandas as pd
import numpy as np

bike_df=pd.read_csv('201907-citibike-tripdata.csv')

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,897,2019-07-01 00:00:00.1320,2019-07-01 00:14:58.0040,493.0,W 45 St & 6 Ave,40.756800,-73.982912,454.0,E 51 St & 1 Ave,40.754557,-73.965930,18340,Subscriber,1966,1
1,267,2019-07-01 00:00:05.1780,2019-07-01 00:04:32.4500,3143.0,5 Ave & E 78 St,40.776321,-73.964274,3226.0,W 82 St & Central Park West,40.782750,-73.971370,21458,Customer,1996,1
2,2201,2019-07-01 00:00:05.2130,2019-07-01 00:36:46.7490,317.0,E 6 St & Avenue B,40.724537,-73.981854,3469.0,India St & West St,40.731814,-73.959950,39874,Subscriber,1986,1
3,1660,2019-07-01 00:00:08.6010,2019-07-01 00:27:48.8050,249.0,Harrison St & Hudson St,40.718710,-74.009001,369.0,Washington Pl & 6 Ave,40.732241,-74.000264,38865,Subscriber,1988,1
4,109,2019-07-01 00:00:12.1580,2019-07-01 00:02:01.5670,3552.0,W 113 St & Broadway,40.805973,-73.964928,3538.0,W 110 St & Amsterdam Ave,40.802692,-73.962950,30256,Subscriber,1997,1


In [132]:
#change column names
bike_df.rename(columns={'tripduration':'duration','starttime':'start','stoptime':'stop','start station name':'start_station',
               'start station id':'start_id','start station latitude':'start_lat',
                'start station longitude':'start_long','end station id':'end_id',
               'end station name':'end_station','end station latitude':'end_lat','end station longitude':'end_long',
               'bikeid':'bike','usertype':'user','birth year':'birth_year'},inplace=True)

In [24]:
bike_df.head()

,duration,start,stop,start_id,start_station,start_lat,start_long,end_id,end_station,end_lat,end_long,bike,user,birth year,gender
0,897,2019-07-01 00:00:00.1320,2019-07-01 00:14:58.0040,493.0,W 45 St & 6 Ave,40.756800,-73.982912,454.0,E 51 St & 1 Ave,40.754557,-73.965930,18340,Subscriber,1966,1
1,267,2019-07-01 00:00:05.1780,2019-07-01 00:04:32.4500,3143.0,5 Ave & E 78 St,40.776321,-73.964274,3226.0,W 82 St & Central Park West,40.782750,-73.971370,21458,Customer,1996,1
2,2201,2019-07-01 00:00:05.2130,2019-07-01 00:36:46.7490,317.0,E 6 St & Avenue B,40.724537,-73.981854,3469.0,India St & West St,40.731814,-73.959950,39874,Subscriber,1986,1
3,1660,2019-07-01 00:00:08.6010,2019-07-01 00:27:48.8050,249.0,Harrison St & Hudson St,40.718710,-74.009001,369.0,Washington Pl & 6 Ave,40.732241,-74.000264,38865,Subscriber,1988,1
4,109,2019-07-01 00:00:12.1580,2019-07-01 00:02:01.5670,3552.0,W 113 St & Broadway,40.805973,-73.964928,3538.0,W 110 St & Amsterdam Ave,40.802692,-73.962950,30256,Subscriber,1997,1


In [30]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2181064 entries, 0 to 2181063
Data columns (total 15 columns):
duration         int64
start            datetime64[ns]
stop             datetime64[ns]
start_id         float64
start_station    object
start_lat        float64
start_long       float64
end_id           float64
end_station      object
end_lat          float64
end_long         float64
bike             int64
user             object
birth year       int64
gender           int64
dtypes: datetime64[ns](2), float64(6), int64(4), object(3)
memory usage: 249.6+ MB


In [38]:
#change column types
bike_df['start']=pd.to_datetime(bike_df['start'])
bike_df['stop']=pd.to_datetime(bike_df['stop'])
bike_df['start_id'].apply(lambda x: str(int(x)) if not pd.isna(x) else x)
bike_df['end_id'].apply(lambda x: str(int(x)) if not pd.isna(x) else x)
bike_df['bike'].apply(lambda x: str(x) if not pd.isna(x) else x)
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2181064 entries, 0 to 2181063
Data columns (total 15 columns):
duration         int64
start            datetime64[ns]
stop             datetime64[ns]
start_id         float64
start_station    object
start_lat        float64
start_long       float64
end_id           float64
end_station      object
end_lat          float64
end_long         float64
bike             int64
user             object
birth year       int64
gender           int64
dtypes: datetime64[ns](2), float64(6), int64(4), object(3)
memory usage: 249.6+ MB


In [187]:
#add day_of_week and hour colummns
bike_df['start_day']=bike_df['start'].dt.weekday
bike_df['end_day']=bike_df['stop'].dt.weekday

bike_df['start_hour']=bike_df['start'].dt.hour
bike_df['end_hour']=bike_df['stop'].dt.hour

#change start_day and end_day to weekday names
weekdays=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
bike_df['start_day']=bike_df['start_day'].apply(lambda x: weekdays[x])
bike_df['end_day']=bike_df['end_day'].apply(lambda x: weekdays[x])

In [150]:
bike_df.head()

,duration,start,stop,start_id,start_station,start_lat,start_long,end_id,end_station,end_lat,end_long,bike,user,birth_year,gender,start_day,end_day,start_hour,end_hour,age_group
0,897,2019-07-01 00:00:00.132,2019-07-01 00:14:58.004,493.0,W 45 St & 6 Ave,40.756800,-73.982912,454.0,E 51 St & 1 Ave,40.754557,-73.965930,18340,Subscriber,1966,1,Monday,Monday,0,0,Old
1,267,2019-07-01 00:00:05.178,2019-07-01 00:04:32.450,3143.0,5 Ave & E 78 St,40.776321,-73.964274,3226.0,W 82 St & Central Park West,40.782750,-73.971370,21458,Customer,1996,1,Monday,Monday,0,0,Young
2,2201,2019-07-01 00:00:05.213,2019-07-01 00:36:46.749,317.0,E 6 St & Avenue B,40.724537,-73.981854,3469.0,India St & West St,40.731814,-73.959950,39874,Subscriber,1986,1,Monday,Monday,0,0,Middle-Aged
3,1660,2019-07-01 00:00:08.601,2019-07-01 00:27:48.805,249.0,Harrison St & Hudson St,40.718710,-74.009001,369.0,Washington Pl & 6 Ave,40.732241,-74.000264,38865,Subscriber,1988,1,Monday,Monday,0,0,Middle-Aged
4,109,2019-07-01 00:00:12.158,2019-07-01 00:02:01.567,3552.0,W 113 St & Broadway,40.805973,-73.964928,3538.0,W 110 St & Amsterdam Ave,40.802692,-73.962950,30256,Subscriber,1997,1,Monday,Monday,0,0,Young


In [151]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange,HoverTool,Legend,LegendItem
from math import pi
from bokeh.palettes import small_palettes

output_notebook()

Loading BokehJS ...

In [152]:
def make_hist_df(series,rnge,bins=50,normalized=False):
    hist,edges=np.histogram(series,bins,rnge)
    if normalized:
            return pd.DataFrame({'top':hist/series.count(),'left':edges[:-1],'right':edges[1:]})
    return pd.DataFrame({'top':hist,'left':edges[:-1],'right':edges[1:]})

def plot_histogram(data_df,legend=None,color='blue',bottom=0,alpha=.3):
    p.quad(left=data_df['left'], right=data_df['right'], top=data_df['top'], bottom=bottom,
           fill_color=color, line_color=None, alpha=.3, legend=legend)

In [165]:
bike_male=bike_df[bike_df['gender']==1]
bike_female=bike_df[bike_df['gender']==2]

hist_df_1=make_hist_df(bike_male['duration']/60,[0,60],60,True)
hist_df_2=make_hist_df(bike_female['duration']/60,[0,60],60,True)

p=figure(title='Trip Length by Gender',plot_height=400,plot_width=800)

plot_histogram(hist_df_1,'Male','orange')
plot_histogram(hist_df_2,'Female')

show(p)

In [154]:
tourist_df=bike_df[bike_df['user']=='Customer']
local_df=bike_df[bike_df['user']=='Subscriber']

hist_df_1=make_hist_df(tourist_df['duration']/60,[0,60],60,True)
hist_df_2=make_hist_df(local_df['duration']/60,[0,60],60,True)

p=figure(title='Trip Length by User Type',plot_height=400,plot_width=800)
plot_histogram(hist_df_1,'Tourist')
plot_histogram(hist_df_2,'Local','orange')
show(p)    

In [155]:
def get_age_group(n):
    if n<1969:
        return 'Old'
    if n<=1989:
        return 'Middle-Aged'
    else:
        return 'Young'

bike_df['age_group']=bike_df['birth_year'].apply(get_age_group)

In [156]:
young_df=bike_df[bike_df['age_group']=='Young']
middle_df=bike_df[bike_df['age_group']=='Middle-Aged']
old_df=bike_df[bike_df['age_group']=='Old']

hist_df_1=make_hist_df(young_df['duration']/60,[0,60],60,True)
hist_df_2=make_hist_df(middle_df['duration']/60,[0,60],60,True)
hist_df_3=make_hist_df(old_df['duration']/60,[0,60],60,True)

p=figure(title='Trip Length by Age Group',plot_height=400,plot_width=800)
plot_histogram(hist_df_1,'Young')
plot_histogram(hist_df_2,'Middle-Aged','red')
plot_histogram(hist_df_3,'Old','green')
show(p)    

Next I look at how trips are distributed by hour of day and by day of week, relative to gender, user type, and age group. The most appropriate chart for this is a stacked or grouped bar chart, but when dealing with hours of day and only two groups (as in the case of gender and user type) I think overlapping bar charts are more suggestive, so I'll use them instead (I think it makes the comparison between subgroups easier).

In [175]:
male_by_day=bike_male.groupby('start_day')['start_id'].count()/bike_male['start_id'].count()
male_by_day=male_by_day.to_frame()
male_by_day.rename(columns={'start_id':'male_count'},inplace=True)
male_by_day.reset_index(inplace=True)
female_by_day=bike_female.groupby('start_day')['start_id'].count()/bike_female['start_id'].count()
female_by_day=female_by_day.to_frame()
female_by_day.rename(columns={'start_id':'female_count'},inplace=True)
female_by_day.reset_index(inplace=True)
weekday_counts_gender_df=pd.merge(male_by_day,female_by_day,on='start_day')
weekday_counts_gender_df

,start_day,male_count,female_count
0,Friday,0.136987,0.133892
1,Monday,0.170140,0.161413
2,Saturday,0.103357,0.120285
3,Sunday,0.102050,0.119225
4,Thursday,0.118064,0.114877
5,Tuesday,0.187940,0.178139
6,Wednesday,0.181462,0.172170


In [201]:
from bokeh.transform import dodge
source=ColumnDataSource(weekday_counts_gender_df)

p=figure(x_range=weekdays,y_range=(0,.3),plot_width=800,plot_height=400)

p.vbar(x=dodge('start_day',-0.2,range=p.x_range),top='male_count',width=.33,source=source,color='blue',legend='Male',alpha=.3)
p.vbar(x=dodge('start_day',0.2,range=p.x_range),top='female_count',width=.33,source=source,color='red',legend='Female',alpha=.3)
p.legend.location = "top_right"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range.range_padding = 0.02

show(p)

In [226]:
tourist_by_day=tourist_df.groupby('start_day')['start_id'].count()/tourist_df['start_id'].count()
tourist_by_day=tourist_by_day.to_frame()
tourist_by_day.rename(columns={'start_id':'tourist_count'},inplace=True)
tourist_by_day.reset_index(inplace=True)
local_by_day=local_df.groupby('start_day')['start_id'].count()/local_df['start_id'].count()
local_by_day=local_by_day.to_frame()
local_by_day.rename(columns={'start_id':'local_count'},inplace=True)
local_by_day.reset_index(inplace=True)
weekday_counts_user_df=pd.merge(tourist_by_day,local_by_day,on='start_day')

Note:the above cell can be simplified by using pandas' pivot_table() function. The first four lines above can be replaces by these two lines (and similar for local_by_day). However, I like practicing groupby and more basic pandas functions so I 

In [224]:
# tourist_by_day=pd.pivot_table(tourist_df,index='start_day',values='user',aggfunc=len)
# tourist_by_day['user']=tourist_by_day['user']/tourist_df['start_id'].count()

1.0000370375269516

In [227]:
source=ColumnDataSource(weekday_counts_user_df)

p=figure(x_range=weekdays,y_range=(0,.3),plot_width=800,plot_height=400)

p.vbar(x=dodge('start_day',-0.2,range=p.x_range),top='tourist_count',width=.33,source=source,color='blue',legend='Tourist',alpha=.3)
p.vbar(x=dodge('start_day',0.2,range=p.x_range),top='local_count',width=.33,source=source,color='red',legend='Local',alpha=.3)
p.legend.location = "top_right"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range.range_padding = 0.02

show(p)

In [144]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2181064 entries, 0 to 2181063
Data columns (total 20 columns):
duration         int64
start            datetime64[ns]
stop             datetime64[ns]
start_id         float64
start_station    object
start_lat        float64
start_long       float64
end_id           float64
end_station      object
end_lat          float64
end_long         float64
bike             int64
user             object
birth_year       int64
gender           int64
start_day        int64
end_day          int64
start_hour       int64
end_hour         int64
age_group        object
dtypes: datetime64[ns](2), float64(6), int64(8), object(4)
memory usage: 332.8+ MB
